In [5]:
# coding=utf-8
# @author:bryan
# blog: https://blog.csdn.net/bryan__
# github: https://github.com/YouChouNoBB/2018-tencent-ad-competition-baseline
import pandas as pd
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os
import gc
import math
import numpy as np

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
ad_feature=pd.read_csv('./Oridata/tc2018/adFeature.csv')
train=pd.read_csv('./Oridata/tc2018/train.csv')
predict=pd.read_csv('./Oridata/tc2018/test1.csv')
train.loc[train['label']==-1,'label']=0
predict['label']=-1

In [8]:
if os.path.exists('./Oridata/tc2018/userFeature.csv'):
    user_feature=pd.read_csv('./Oridata/tc2018/userFeature.csv')
'''
else:
    print("-----------------------------")
    userFeature_data = []
    with open('./Oridata/tc2018/userFeature.data', 'r') as f:
        cnt = 0
        for i, line in enumerate(f):
            line = line.strip().split('|')
            userFeature_dict = {}
            for each in line:
                each_list = each.split(' ')
                userFeature_dict[each_list[0]] = ' '.join(each_list[1:])
            userFeature_data.append(userFeature_dict)
            if i % 100000 == 0:
                print(i)
            if i % 1000000 == 0:
                user_feature = pd.DataFrame(userFeature_data)
                user_feature.to_csv('../data/userFeature_' + str(cnt) + '.csv', index=False)
                cnt += 1
                del userFeature_data, user_feature
                userFeature_data = []
        user_feature = pd.DataFrame(userFeature_data)
        user_feature.to_csv('../data/userFeature_' + str(cnt) + '.csv', index=False)
        del userFeature_data, user_feature
        user_feature = pd.concat(
            [pd.read_csv('../data/userFeature_' + str(i) + '.csv') for i in range(cnt + 1)]).reset_index(drop=True)
        user_feature.to_csv('./dataAfterProcess16/userFeature.csv', index=False)
'''

'\nelse:\n    print("-----------------------------")\n    userFeature_data = []\n    with open(\'./Oridata/tc2018/userFeature.data\', \'r\') as f:\n        cnt = 0\n        for i, line in enumerate(f):\n            line = line.strip().split(\'|\')\n            userFeature_dict = {}\n            for each in line:\n                each_list = each.split(\' \')\n                userFeature_dict[each_list[0]] = \' \'.join(each_list[1:])\n            userFeature_data.append(userFeature_dict)\n            if i % 100000 == 0:\n                print(i)\n            if i % 1000000 == 0:\n                user_feature = pd.DataFrame(userFeature_data)\n                user_feature.to_csv(\'../data/userFeature_\' + str(cnt) + \'.csv\', index=False)\n                cnt += 1\n                del userFeature_data, user_feature\n                userFeature_data = []\n        user_feature = pd.DataFrame(userFeature_data)\n        user_feature.to_csv(\'../data/userFeature_\' + str(cnt) + \'.csv\', index

In [9]:
user_feature.shape

(11420039, 24)

In [10]:
def LGB_predict(train_x,train_y,test_x,res,index):
    print("LGB test")
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
        max_depth=-1, n_estimators=1500, objective='binary',
        subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
        learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=-1
    )
    clf.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='auc',early_stopping_rounds=100)
    res['score'+str(index)] = clf.predict_proba(test_x)[:,1]
    res['score'+str(index)] = res['score'+str(index)].apply(lambda x: float('%.6f' % x))
    print(str(index)+' predict finish!')
    res = res.reset_index(drop=True)
    print(res.head())
    gc.collect()
    return res['score'+str(index)]

In [11]:
def batch_predict(slice,index):
    data=pd.concat([slice,predict])
    data=pd.merge(data,ad_feature,on='aid',how='left')
    data=pd.merge(data,user_feature,on='uid',how='left')
    data=data.fillna('-1')
    one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
           'adCategoryId', 'productId', 'productType']
    vector_feature=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5','kw1','kw2','kw3','topic1','topic2','topic3']
    for feature in one_hot_feature:
        try:
            data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
        except:
            data[feature] = LabelEncoder().fit_transform(data[feature])
    train=data[data.label!=-1]
    train_y=train.pop('label')
    test=data[data.label==-1]
    res=test[['aid','uid']]
    test=test.drop('label',axis=1)
    enc = OneHotEncoder()
    train_x=train[['creativeSize']]
    test_x=test[['creativeSize']]
    for feature in one_hot_feature:
        enc.fit(data[feature].values.reshape(-1, 1))
        train_a=enc.transform(train[feature].values.reshape(-1, 1))
        test_a = enc.transform(test[feature].values.reshape(-1, 1))
        train_x= sparse.hstack((train_x, train_a))
        test_x = sparse.hstack((test_x, test_a))
        print(feature+' finish')
    print('one-hot prepared !')
    cv=CountVectorizer()
    for feature in vector_feature:
        cv.fit(data[feature])
        train_a = cv.transform(train[feature])
        test_a = cv.transform(test[feature])
        train_x = sparse.hstack((train_x, train_a))
        test_x = sparse.hstack((test_x, test_a))
        print(feature + ' finish')
    print('cv prepared !')
    return LGB_predict(train_x, train_y, test_x, res,index)

In [12]:
#数据分片处理，对每片分别训练预测，然后求平均
cnt=20
size = math.ceil(len(train) / cnt)
result=[]
for i in range(cnt):
    start = size * i
    end = (i + 1) * size if (i + 1) * size < len(train) else len(train)
    slice = train[start:end]
    result.append(batch_predict(slice,i))
    gc.collect()

result=pd.concat(result,axis=1)
result['score']=np.mean(result,axis=1)
result=result.reset_index(drop=True)
result=pd.concat([predict[['aid','uid']].reset_index(drop=True),result['score']],axis=1)
result['score'] = result['score'].apply(lambda x: float('%.6f' % x))
result.to_csv('./dataAfterProcess16/submission.csv', index=False)
os.system('zip ./dataAfterProcess16/baseline.zip ./dataAfterProcess16/submission.csv')

LBS finish
age finish
carrier finish
consumptionAbility finish
education finish
gender finish
house finish
os finish
ct finish
marriageStatus finish
advertiserId finish
campaignId finish
creativeId finish
adCategoryId finish
productId finish
productType finish
one-hot prepared !
appIdAction finish
appIdInstall finish
interest1 finish
interest2 finish
interest3 finish
interest4 finish
interest5 finish
kw1 finish
kw2 finish
kw3 finish
topic1 finish
topic2 finish
topic3 finish
cv prepared !
LGB test
[1]	valid_0's auc: 0.647779	valid_0's binary_logloss: 0.191702
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.652705	valid_0's binary_logloss: 0.19085
[3]	valid_0's auc: 0.659065	valid_0's binary_logloss: 0.190136
[4]	valid_0's auc: 0.663024	valid_0's binary_logloss: 0.189483
[5]	valid_0's auc: 0.664396	valid_0's binary_logloss: 0.188853
[6]	valid_0's auc: 0.665574	valid_0's binary_logloss: 0.188394
[7]	valid_0's auc: 0.665475	valid_0's binary_logloss: 0.18

[122]	valid_0's auc: 0.716075	valid_0's binary_logloss: 0.177012
[123]	valid_0's auc: 0.716272	valid_0's binary_logloss: 0.176985
[124]	valid_0's auc: 0.716438	valid_0's binary_logloss: 0.176946
[125]	valid_0's auc: 0.716627	valid_0's binary_logloss: 0.176917
[126]	valid_0's auc: 0.71696	valid_0's binary_logloss: 0.17688
[127]	valid_0's auc: 0.717232	valid_0's binary_logloss: 0.176845
[128]	valid_0's auc: 0.717432	valid_0's binary_logloss: 0.176814
[129]	valid_0's auc: 0.717636	valid_0's binary_logloss: 0.176778
[130]	valid_0's auc: 0.717839	valid_0's binary_logloss: 0.176749
[131]	valid_0's auc: 0.718044	valid_0's binary_logloss: 0.176717
[132]	valid_0's auc: 0.718257	valid_0's binary_logloss: 0.176693
[133]	valid_0's auc: 0.718454	valid_0's binary_logloss: 0.176666
[134]	valid_0's auc: 0.718692	valid_0's binary_logloss: 0.176639
[135]	valid_0's auc: 0.718909	valid_0's binary_logloss: 0.176603
[136]	valid_0's auc: 0.719109	valid_0's binary_logloss: 0.176573
[137]	valid_0's auc: 0.7193

[249]	valid_0's auc: 0.736837	valid_0's binary_logloss: 0.173958
[250]	valid_0's auc: 0.736968	valid_0's binary_logloss: 0.173939
[251]	valid_0's auc: 0.737078	valid_0's binary_logloss: 0.173922
[252]	valid_0's auc: 0.737184	valid_0's binary_logloss: 0.173906
[253]	valid_0's auc: 0.737301	valid_0's binary_logloss: 0.17389
[254]	valid_0's auc: 0.7375	valid_0's binary_logloss: 0.173863
[255]	valid_0's auc: 0.737612	valid_0's binary_logloss: 0.173844
[256]	valid_0's auc: 0.737776	valid_0's binary_logloss: 0.173825
[257]	valid_0's auc: 0.737908	valid_0's binary_logloss: 0.173806
[258]	valid_0's auc: 0.738002	valid_0's binary_logloss: 0.173794
[259]	valid_0's auc: 0.738149	valid_0's binary_logloss: 0.173778
[260]	valid_0's auc: 0.73825	valid_0's binary_logloss: 0.173752
[261]	valid_0's auc: 0.73834	valid_0's binary_logloss: 0.173737
[262]	valid_0's auc: 0.738452	valid_0's binary_logloss: 0.173715
[263]	valid_0's auc: 0.73863	valid_0's binary_logloss: 0.173692
[264]	valid_0's auc: 0.738809	v

[378]	valid_0's auc: 0.751659	valid_0's binary_logloss: 0.171682
[379]	valid_0's auc: 0.751736	valid_0's binary_logloss: 0.171669
[380]	valid_0's auc: 0.751878	valid_0's binary_logloss: 0.171652
[381]	valid_0's auc: 0.751931	valid_0's binary_logloss: 0.171641
[382]	valid_0's auc: 0.752084	valid_0's binary_logloss: 0.171621
[383]	valid_0's auc: 0.752181	valid_0's binary_logloss: 0.171609
[384]	valid_0's auc: 0.752272	valid_0's binary_logloss: 0.171591
[385]	valid_0's auc: 0.752345	valid_0's binary_logloss: 0.171577
[386]	valid_0's auc: 0.752497	valid_0's binary_logloss: 0.171552
[387]	valid_0's auc: 0.75259	valid_0's binary_logloss: 0.171534
[388]	valid_0's auc: 0.752728	valid_0's binary_logloss: 0.171512
[389]	valid_0's auc: 0.752838	valid_0's binary_logloss: 0.171495
[390]	valid_0's auc: 0.752944	valid_0's binary_logloss: 0.171474
[391]	valid_0's auc: 0.753059	valid_0's binary_logloss: 0.17146
[392]	valid_0's auc: 0.75319	valid_0's binary_logloss: 0.171442
[393]	valid_0's auc: 0.75329

[507]	valid_0's auc: 0.763945	valid_0's binary_logloss: 0.169722
[508]	valid_0's auc: 0.764082	valid_0's binary_logloss: 0.169704
[509]	valid_0's auc: 0.764188	valid_0's binary_logloss: 0.16969
[510]	valid_0's auc: 0.764304	valid_0's binary_logloss: 0.169674
[511]	valid_0's auc: 0.764391	valid_0's binary_logloss: 0.169663
[512]	valid_0's auc: 0.764476	valid_0's binary_logloss: 0.169649
[513]	valid_0's auc: 0.764571	valid_0's binary_logloss: 0.169638
[514]	valid_0's auc: 0.764696	valid_0's binary_logloss: 0.169622
[515]	valid_0's auc: 0.764815	valid_0's binary_logloss: 0.169606
[516]	valid_0's auc: 0.764907	valid_0's binary_logloss: 0.169588
[517]	valid_0's auc: 0.764987	valid_0's binary_logloss: 0.169575
[518]	valid_0's auc: 0.765077	valid_0's binary_logloss: 0.169561
[519]	valid_0's auc: 0.765191	valid_0's binary_logloss: 0.169544
[520]	valid_0's auc: 0.765272	valid_0's binary_logloss: 0.169531
[521]	valid_0's auc: 0.765406	valid_0's binary_logloss: 0.169512
[522]	valid_0's auc: 0.765

...

[1121]	valid_0's auc: 0.809274	valid_0's binary_logloss: 0.161797
[1122]	valid_0's auc: 0.809321	valid_0's binary_logloss: 0.161785
[1123]	valid_0's auc: 0.809376	valid_0's binary_logloss: 0.161772
[1124]	valid_0's auc: 0.809392	valid_0's binary_logloss: 0.161766
[1125]	valid_0's auc: 0.809434	valid_0's binary_logloss: 0.161755
[1126]	valid_0's auc: 0.809501	valid_0's binary_logloss: 0.161743
[1127]	valid_0's auc: 0.809581	valid_0's binary_logloss: 0.161731
[1128]	valid_0's auc: 0.809661	valid_0's binary_logloss: 0.161716
[1129]	valid_0's auc: 0.8097	valid_0's binary_logloss: 0.16171
[1130]	valid_0's auc: 0.809712	valid_0's binary_logloss: 0.161701
[1131]	valid_0's auc: 0.809759	valid_0's binary_logloss: 0.161693
[1132]	valid_0's auc: 0.809817	valid_0's binary_logloss: 0.161683
[1133]	valid_0's auc: 0.809852	valid_0's binary_logloss: 0.16167
[1134]	valid_0's auc: 0.809933	valid_0's binary_logloss: 0.161658
[1135]	valid_0's auc: 0.809957	valid_0's binary_logloss: 0.161652
[1136]	valid_0

[1248]	valid_0's auc: 0.815806	valid_0's binary_logloss: 0.160416
[1249]	valid_0's auc: 0.815854	valid_0's binary_logloss: 0.160406
[1250]	valid_0's auc: 0.815908	valid_0's binary_logloss: 0.160399
[1251]	valid_0's auc: 0.815957	valid_0's binary_logloss: 0.160385
[1252]	valid_0's auc: 0.816024	valid_0's binary_logloss: 0.160373
[1253]	valid_0's auc: 0.81607	valid_0's binary_logloss: 0.160363
[1254]	valid_0's auc: 0.816126	valid_0's binary_logloss: 0.160352
[1255]	valid_0's auc: 0.816174	valid_0's binary_logloss: 0.160343
[1256]	valid_0's auc: 0.816205	valid_0's binary_logloss: 0.160335
[1257]	valid_0's auc: 0.816241	valid_0's binary_logloss: 0.160326
[1258]	valid_0's auc: 0.816288	valid_0's binary_logloss: 0.160316
[1259]	valid_0's auc: 0.816348	valid_0's binary_logloss: 0.160308
[1260]	valid_0's auc: 0.816399	valid_0's binary_logloss: 0.160295
[1261]	valid_0's auc: 0.816428	valid_0's binary_logloss: 0.160286
[1262]	valid_0's auc: 0.8165	valid_0's binary_logloss: 0.16027
[1263]	valid_0

[1375]	valid_0's auc: 0.822425	valid_0's binary_logloss: 0.159017
[1376]	valid_0's auc: 0.822479	valid_0's binary_logloss: 0.159008
[1377]	valid_0's auc: 0.822498	valid_0's binary_logloss: 0.158997
[1378]	valid_0's auc: 0.822548	valid_0's binary_logloss: 0.158986
[1379]	valid_0's auc: 0.822618	valid_0's binary_logloss: 0.158975
[1380]	valid_0's auc: 0.822671	valid_0's binary_logloss: 0.158967
[1381]	valid_0's auc: 0.822703	valid_0's binary_logloss: 0.15896
[1382]	valid_0's auc: 0.822777	valid_0's binary_logloss: 0.158949
[1383]	valid_0's auc: 0.822817	valid_0's binary_logloss: 0.15894
[1384]	valid_0's auc: 0.822855	valid_0's binary_logloss: 0.15893
[1385]	valid_0's auc: 0.82291	valid_0's binary_logloss: 0.158914
[1386]	valid_0's auc: 0.822957	valid_0's binary_logloss: 0.158905
[1387]	valid_0's auc: 0.823015	valid_0's binary_logloss: 0.158893
[1388]	valid_0's auc: 0.823043	valid_0's binary_logloss: 0.158885
[1389]	valid_0's auc: 0.823078	valid_0's binary_logloss: 0.158876
[1390]	valid_0

[1500]	valid_0's auc: 0.828646	valid_0's binary_logloss: 0.157678
Did not meet early stopping. Best iteration is:
[1500]	valid_0's auc: 0.828646	valid_0's binary_logloss: 0.157678
19 predict finish!
    aid       uid   score19
0  2118  64355836  0.019608
1   692  45051997  0.030294
2   692  10869198  0.035789
3  1918  75929554  0.026914
4  1596   5790162  0.281719


32512